### Keyword arguments 
These are very useful when we need to accept arbitrary arguments to pass to  a second function, but we don't know what those arguments will be, for example when building support for multiple inheritance. One option is to combine the variable argument and variable keyword argument syntax in one function call, and use normal positional and default arguments as well. The following example below, although somewhat contrived, demonstrates the four types in action (Phillips, Dusty, "Python Object-Oriented Programming", third edition, p 211.)

In [14]:
import shutil
import os.path

def augmented_move(target_folder,
                   *filenames,
                   verbose=False,
                   **specific):
    """Move all filenames into the target_folder, allowing foe specific treatment 
    of certain file types"""
    
    def print_verbose(message, filenames):
        """Print the message only when verbose is enables"""
        if verbose:
            print(message.format(filename))
            
    for filename in filenames:
        target_path = os.path.join(target_folder, filename)
        if filename in specific:
            if specific[filename] == "ignore":
                print_verbose("Ignoring {0} ", filename)
            elif specific[filename] == "copy":
                print_verbose("Copying {0} ", filename)
                shutil.copyfile(filename, target_path)
            else:
                print_verbose("Moving {0} ", filename)
                shutil.move(filename, target_path)
            

In [15]:
augmented_move('move here', 'three', verbose=True, four='copy')

### Functions are objects too
Should be able to pass two different functions into a thirsd function and get different output for each one

In [16]:
def my_function():
    print("The first my_function() was called")
my_function.description = "My first silly finction"

def the_second_function():
    print("Now the second fucntion was called")
the_second_function.description = "set description for the_seconf_function"

def another_function(function):
    print("The description:", end=" ")
    print(function.description)
    print("The name: ", end=" ")
    print(function.__name__)
    print("The class: ", end=" ")
    print(function.__class__)
    print("Now call the function that was passed in")
    function()

In [17]:
another_function(my_function)

The description: My first silly finction
The name:  my_function
The class:  <class 'function'>
Now call the function that was passed in
The first my_function() was called


In [18]:
another_function(the_second_function)

The description: set description for the_seconf_function
The name:  the_second_function
The class:  <class 'function'>
Now call the function that was passed in
Now the second fucntion was called


### Event-driven timer
Functions are top-level objects and can be passed around to be executed at a later date, for example , when a certain condition has been satisfied.

In [21]:
import time
import datetime

class TimedEvent:
    def __init__(self, endtime, callback):
        self.endtime = endtime
        self.callback = callback
    
    def ready(self):
        return self.endtime <= datetime.datetime.now()
    
class Timer:
    def __init__(self):
        self.events = []
        
    def call_after(self, delay, callback):
        end_time = datetime.datetime.now() + datetime.timedelta(seconds=delay)
        self.events.append(TimedEvent(end_time, callback))
        
    def run(self):
        while True: #This needs to be interrupted with Ctrl + C
            ready_events = (e for e in self.events if e.ready())
            for event in ready_events:
                event.callback(self)
                self.events.remove(event)
            time.sleep(0.5)
            
# Several callbacks to test the timer below:
def format_time(message, *args):
    now = datetime.datetime.now()
    print(f"{now:%I:%M:%S}: {message}")

def one(timer):
    format_time("Called One")

def two(timer):
    format_time("Called Two")
    
def three(timer):
    format_time("Called Three")

class Repeater:
    def __init__(self):
        self.count = 0
    
    def repeater(self, timer):
        format_time(f"repeat {self.count}" )
        self.count += 1
        timer.call_after(5, self.repeater)

**Note:** the function is passed around like any other object and the timer never needs to know or care what the original name of the function is or where it was defined. When it's time to call the function, the timer simply applies the parenthesis syntax to the stored variable.

In [ ]:
timer = Timer()
timer.call_after(1, one)
timer.call_after(2, one)
timer.call_after(2, two)
timer.call_after(4, two)
timer.call_after(3, three)
timer.call_after(6, three)
repeater = Repeater()
timer.call_after(5, repeater.repeater)
format_time("Starting")
timer.run()

07:12:15: Starting
07:12:16: Called One
07:12:17: Called One
07:12:18: Called Two
07:12:18: Called Three
07:12:19: Called Two
07:12:20: repeat 0
07:12:21: Called Three
07:12:25: repeat 1
07:12:30: repeat 2
07:12:35: repeat 3
07:12:40: repeat 4
07:12:45: repeat 5
07:12:50: repeat 6
07:12:55: repeat 7
07:13:00: repeat 8
07:13:05: repeat 9
07:13:10: repeat 10
07:13:15: repeat 11
07:13:20: repeat 12
07:13:25: repeat 13
07:13:30: repeat 14
07:13:35: repeat 15
07:13:40: repeat 16
07:13:45: repeat 17
07:13:50: repeat 18
07:13:56: repeat 19
07:14:01: repeat 20
07:14:06: repeat 21
07:14:11: repeat 22
07:14:16: repeat 23
07:14:21: repeat 24
07:14:26: repeat 25
07:14:31: repeat 26
07:14:36: repeat 27
07:14:41: repeat 28
07:14:46: repeat 29
07:14:51: repeat 30
07:14:56: repeat 31
07:15:01: repeat 32
07:15:06: repeat 33
07:15:11: repeat 34
07:15:16: repeat 35


### Using functions as attributes
Functions can be set as callable attributes on other objects. It is possible o add or change a function to an instantiated object, demonstrated as follows below:

In [3]:
class A:
    def print(self):
        print("my class is A")

def fake_print():
    print("my class is not A")
    
a = A()
a.print()

my class is A


In [5]:
a.print = fake_print
a.print()

my class is not A
